#Zadanie 2 (7 pkt)
Celem zadania jest rozwiązanie zadania optymalizacji przy pomocy algorytmu genetycznego. Należy znaleźć minimum zadanej funkcji:
\begin{equation}
f(x) = 1.5-\exp \left\{-x_1^2-x_2^2\right\}-0.5 \exp \left\{-\left(x_1-1\right)^2-\left(x_2+2\right)^2\right\}
\end{equation}
Należy wykorzystać mechanizmy krzyżowania punktowego, mutacji oraz selekcji turniejowej. Proszę wykorzystać przygotowany szkielet i zaimplementować niezbędne metody. Opracowane oprogramowanie powinno być uniwersalne - wymiar funkcji, której minimum szukamy może być dowolny (mechanizm *args). Punktacja wygląda następująco:

*   Stworzenie obiektu klasy *Chromosome* z polem *array*, które jest wektorem aktywnych i nieaktywnych genów - **0.5 pkt**
*   Implementacja metody *decode*, która dekoduje część chromosomu (określoną przez początek (*lower_bound*) i koniec (*upper_bound*)) do postaci liczby rzeczywistej. *aoi* to zakres wartości zdekodowanej liczby rzeczywistej. Przykład: liczba 135 w postaci binarnej zapisana przy użyciu 8 bitów to 1000 0111, jeśli nasze *aoi* to [0, 1], to 135 sprowadzone do tego zakresu to 0.529. Można skorzystać z funkcji pomocniczej *min_max_norm* - **1 pkt**
*   Implementacja metody *mutation*, która przyjmuje jako argument prawdopodobieństo mutacji i zmienia wartość jedego, losowego genu na przeciwną - **0.5 pkt**
*   Implementacja metody *crossover*, która wykonuje operację krzyżowania jednopunktowego - **1 pkt**
*   Implementacja metody *eval_objective_func*, która dekoduje cały chromosom (otrzymuje się argumenty funkcji) oraz zwraca wartość funkcji celu dla tych argumentów - **1 pkt**
*   Implementacja metody *tournament_selection*, która przeprowadza selekcję turniejową - **1 pkt**
*   Implementacja metody *reproduce*, która generuje nowe pokolenie - z pewnym prawdopodobieństwem przeprowadza krzyżowanie jednopunktowe lub "przerzuca" rodziców do nowego pokolenia - **0.5 pkt**
*   Implementacja metody *run*, która wykonuje cały alorytm genetyczny dla określonej liczby pokoleń. W każdym pokoleniu należy zapisać dane osobnika z najlepszym chromosomem zdekodowane wartości x i y oraz wartość funkcji strat dla tego osobnika - **0.5 pkt**
*   Proszę, podobnie jak miało to miejsce w przypadku metody gradientowej z poprzednich zajęć, wygenerować wykres przy użyciu funkcji *plot_func* (w przypadku innego typu argumentu *trace*, funkcję można zmodyfikować. Wykres powinien przedstawiać funkcję, której minimum poszukujemy oraz punkty odpowiadające najlepszym osobnikom w danych generacjach, których kolor jest coraz jaśniejszy wraz ze zbliżaniem się do minimum. Proszę zapisać swoje wnioski, w szczególności w odniesieniu do metody gradientowej. - **1 pkt**


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from random import getrandbits

In [14]:
def min_max_norm(val, min_val, max_val, new_min, new_max):
  return (val - min_val) * (new_max - new_min) / (max_val - min_val) + new_min

In [15]:
class Chromosome:
  def __init__(self, length, array=None): # if array is None it should be initialized with random binary vector
    if array is None:
      array = np.random.randint(2, size=length)
    self.array = np.array(array)
    self.length = length

  def decode(self, lower_bound, upper_bound, aoi): # [lower_bound, upper_bound]
    sum = np.sum([2**i for i in range(upper_bound - lower_bound, -1, -1)] * self.array[lower_bound : upper_bound+1])
    max_sum = 2**(upper_bound-lower_bound + 1) - 1
    return (sum / max_sum) * aoi


  def mutation(self, probability):
    self.array = [self.array[i] if np.random.random() > probability else (self.array[i] + 1) % 2 for i in range(self.length)]

  def crossover(self, other: "Chromosome"):
    point = np.random.randint(0, self.length+1) # numbers of genes that will be inherited from the first section
    first_child = np.concatenate((self.array[0: point], other.array[point:]))
    second_child =  np.concatenate((other.array[0: point], self.array[point:]))
    self.array = first_child
    other.array = second_child

In [16]:
import ipytest
ipytest.autoconfig()

def test_init():
    c = Chromosome(16)
    assert len(c.array) == 16
    assert c.length == 16

def test_decode():
    c = Chromosome(8, [1, 0, 1, 1, 0, 1, 1, 1]) # 11, 7
    assert c.decode(0, 3, 15) == 11
    assert c.decode(4, 7, 15) == 7
    assert c.decode(0, 0, 1) == 1

def test_mutation():
    c = Chromosome(8, [1, 0, 1, 1, 0, 1, 1, 1])
    c.mutation(1)
    assert c.array[0] == 0
    assert c.array[1] == 1

def test_crossover():
    c1 = Chromosome(4, [1, 0, 1, 1])
    c2 = Chromosome(4, [0, 1, 0, 1])
    c1.crossover(c2)
    assert c1.length == 4
    assert c2.length == 4



In [17]:
class GeneticAlgorithm:
  def __init__(self, chromosome_length, obj_func_num_args, objective_function, aoi, population_size=1000,
               tournament_size=2, mutation_probability=0.05, crossover_probability=0.8, num_steps=30):
    assert chromosome_length % obj_func_num_args == 0, "Number of bits for each argument should be equal"
    self.chromosome_lengths = chromosome_length
    self.obj_func_num_args = obj_func_num_args
    self.bits_per_arg = int(chromosome_length / obj_func_num_args)
    self.objective_function = objective_function
    self.aoi = aoi
    self.tournament_size = tournament_size
    self.mutation_probability = mutation_probability
    self.crossover_probability = crossover_probability
    self.num_steps = num_steps
    self.population_size = population_size
    self.population = [Chromosome(chromosome_length) for i in range(population_size)]

  def eval_objective_func(self, chromosome: "Chromosome"):
    n_of_bits = chromosome.length // self.obj_func_num_args
    return self.objective_function([chromosome.decode(i * n_of_bits, i * n_of_bits + n_of_bits - 1, self.aoi) for i in range(self.obj_func_num_args)])

  def tournament_selection(self):
    self.population = [max(np.random.choice(self.population, self.tournament_size, replace=True),key=lambda choice: self.eval_objective_func(choice)) for i in range(self.population_size)]


  def reproduce(self, parents):
    parents[0].crossover(parents[1])

  def plot_func(self, trace):
    X = np.arange(-2, 3, 0.05)
    Y = np.arange(-4, 2, 0.05)
    X, Y = np.meshgrid(X, Y)
    Z = 1.5 - np.exp(-X ** (2) - Y ** (2)) - 0.5 * np.exp(-(X - 1) ** (2) - (Y + 2) ** (2))
    plt.figure()
    plt.contour(X, Y, Z, 10)
    cmaps = [[ii / len(trace), 0, 0] for ii in range(len(trace))]
    plt.scatter([x[0] for x in trace], [x[1] for x in trace], c=cmaps)
    plt.show()

  def run(self):
    pass

In [18]:
def test_eval_objective_func():
    ga = GeneticAlgorithm(8, 2, lambda arr: arr[0] + arr[1], 15)
    c = Chromosome(length=8, array=[1,1,1,1, 0,1,0,1]) # 15 + 5 = 20
    assert ga.eval_objective_func(c) == 20

def test_eval_objective_func2():
    ga = GeneticAlgorithm(9, 3, lambda arr: arr[0] * arr[1] + arr[2], 7)
    c = Chromosome(length=9, array=[1, 0, 1, 1, 1, 0, 0, 0, 1]) # 5 * 6 + 1 = 31
    assert ga.eval_objective_func(c) == 31




ipytest.run()

.

.....                                                                                       [100%]
6 passed in 0.02s


<ExitCode.OK: 0>